In [33]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 
from typing import List, Tuple, Dict, Optional, Union

import qube
%qubed

%load_ext autoreload
%autoreload 2

import tests.qube.ta.utils_for_testing as test
from qube.utils import reload_pyx_module
from dataclasses import dataclass

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
reload_pyx_module('../src/qube/core/')
from qube.core.series import TimeSeries; del sys.modules['qube.core.series']
from qube.core.series import (TimeSeries, Bar, sma, ema, tema, dema, kama, recognize_time, OHLCV, RollingSum, lag, compare)

	>>> [dev] : module series reloaded


# Strategy high level design

In [3]:
@dataclass
class Instrument:
    symbol: str
    exchange: str

    # tick size
    tick_size: float

    # true for futures
    is_futures: bool

    # futures contract size
    futures_contract_size: float = 1

    # instrument used for conversion to main basis
    # let's say we trade BTC/ETH with main account in USDT
    # so we need to use ETH/USDT for convert profits/losses to USDT
    currency_conversion_instrument: 'Instrument' = None

class Trade:
    time: int
    price: float
    size: float

class Quote:
    time: int
    bid: float
    ask: float
    bid_size: float
    ask_size: float

class IStrategy:
    def __init__(self, **kwargs) -> None:
        print("Here")
        self.populate_parameters(**kwargs)

    def populate_parameters(self, **kwargs):
        for k,v in kwargs.items():
            if k.startswith('_'):
                raise ValueError("Internal variable can't be set from external parameter !")
            if hasattr(self, k):
                self.__dict__[k] = v

    def instruments(self) -> List[Instrument]:
        pass

    def position(self, position: float, group=None, stop=None, take=None, limit=None, cancel=False):
        print(f"NEW POSITION: {position} [{group}]")
        pass

    def cancel(self, group=None):
        print(f"CANCEL: all from [{group}]")

    def open_position(self, instrument: Instrument) -> float:
        """
        Returns open position
        """
        return 0.0
        
class PositionTracker:
    pass
class CapitalManager:
    pass
class FixedCapitalPercentage(CapitalManager):
    pass
class AtrTracker(PositionTracker):
    pass

In [32]:
class CrossTestClassical(IStrategy):
    period_fast: int = 5
    period_slow: int = 50

    def fit(self):
        for i in self.Instruments():
            ohlc = self.ohlc(i, '1H')
            # do the model trainig stuff ...
        return self

    def process_data(self, timestamp: np.datetime64, instrument: Optional[Instrument]):
        ohlc = self.ohlc(instrument, '1H')
        # tick_bars = self.tickbars(instrument, 200) # returns series of 200 tick bars
        # trades = self.trades(instrument) # returns series of trades
        # quotes = self.quotes(instrument) # returns series of quotes
        fast = ema(ohlc.close, self.period_fast)
        slow = ema(ohlc.close, self.period_slow)
        bar:Bar = ohlc[0]

        if (fast[0] > slow[0]) and (fast[1] < slow[1]):
            return instrument.position(+1, 'long', stop=bar.low, cancel=True)
        elif (fast[0] < slow[0]) and (fast[1] > slow[1]):
            return instrument.position(-1, 'short', stop=bar.high, cancel=True)

    def tracker(self) -> Optional[PositionTracker]:
        return None

    def capital_manager(self) -> Optional[CapitalManager]:
        return None 

In [21]:
CrossTestClassical()

Here


In [9]:
class PairsTrading(IStrategy):
    period: int = 15
    timeframe: str = '1H'
    __pair_for: Dict[Instrument, Instrument] = {}

    def fit(self, instrument: Instrument):
        closes = self.prices(self.timeframe, 'close', self.instruments()).pd()
        # - find pairs
        self.pairs_finder(closes, self.period, self.instruments())
        return self

    def pairs_finder(self, period, closes: pd.DataFrame):
        self.__pair_for.clear()
        self.__pair_for[self.instruments()[0]] = self.instruments()[1]

    def process_data(self, timestamp: np.datetime64, instrument1: Optional[Instrument]):
        instrument2 = self.__pair_for[instrument1]
        delta = self.ohlc(instrument1, self.timeframe).close - self.ohlc(instrument2, self.timeframe).close, 
        spread = (delta - ema(delta, self.period)) / std(delta - ema(delta, self.period)) 
        if spread[0] > 2:
            return instrument1.position(-1, 'short'), instrument2.position(+1, 'long')
        elif spread[0] < -2:
            return instrument2.position(-1, 'short'), instrument1.position(+1, 'long')

    def tracker(self, instrument: Instrument) -> Optional[PositionTracker]:
        return None

    def capital_manager(
            self, 
            instruments: List[Instrument], 
            available_capital: float, 
            entry_prices: List[float], stop_prices: List[float], take_prices: List[float]
        ) -> Optional[CapitalManager]:
        return None 

In [ ]:
s1 = CrossTestClassical(period_fast=444)

In [ ]:
m = Market('binance.um.usdt.vip0')

In [31]:
data1Min = OHLCV('15Min')

In [ ]:
m.backtest(
    {
        'Signals': CrossTestClassical(period_fast=12),
        'CapPct25%': CrossTestClassical(period_fast=12) + FixedCapitalPercentage(capital_in_risk=0.25),
        'AtrTracker2': CrossTestClassical(period_fast=12) + AtrTracker(stop_atrs=2, initial_bar=0, period=200),
    },
    capital_base=100000,
    leverage=5,
    start='2021-01-01', end='now',
    fit_start='2021-01-01', fit_end='now',

    start='2021-01-01', end='',
    fit_start='2021-01-01', fit_end='+4w',

    data = data1Min.resample('15Min'),   # data to feed
    handle_on='closes',                     # default on data's timeframe close prices
    handle_on='closes.1H',                  # on 1H bar's close price
    handle_on='quotes',                     # on every quote
    handle_on='trades',                     # on every trade
    handle_on='trades,quotes,15Min.closes', # on every trade or quote or close
    handle_on='59Min59S',                   # on timer every T
    symbols=['BTCUSDT', 'ETHUSDT', 'ETHUSDT']
)

In [ ]:
'1H - 5S'

In [ ]:
# # from zipline
# result = run_algorithm(start=start.tz_localize('UTC'),
#                        end=end.tz_localize('UTC'),
#                        initialize=initialize,
#                        handle_data=handle_data,
#                        capital_base=100000,
#                        benchmark_returns=benchmark_returns,
#                        bundle='quandl',
#                        data_frequency='daily')